In [54]:
import pandas as pd
import category_encoders as ce
from xgboost import XGBRFRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.metrics import root_mean_squared_log_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy.stats import zscore
from geopy.distance import geodesic


df1 = pd.read_csv(r'data/all_v2.csv') # 2661

In [ ]:
# Для предсказания

my_house = {
    'building_type': [1],
    'level': [5],
    'levels': [17],
    'rooms':[3],
    'area': [66.6],
    'kitchen_area': [13.8],
    'object_type': [1],
    'geo_lat': [59.985878],
    'geo_lon': [30.238066],
    'price': [16666666],
    'date': ['2025-03-11'],
    'time': ['21:00:00'],
    'region': [2661]
}

my_house = pd.DataFrame(my_house)

In [56]:
df1 = pd.concat([df1, my_house], axis=0).reset_index()

In [57]:
df1_1 = df1[(df1['region'] == 2661) &
             (df1['area'] >= 20) &
               (df1['area'] <= 200) &
                 (df1['kitchen_area'] >= 6) &
                 (df1['kitchen_area'] <= 30) &
                 (df1['price'] >= 1500000) &
                 (df1['price'] <= 50000000)]

df1_1['rooms'] = df1_1['rooms'].replace({-1: 0, -2: 0})
df1_1['height'] = df1_1['level'] / df1_1['levels']
df1_1 = df1_1[df1_1['height'] <= 1]
df1_1['ratio'] = df1_1['kitchen_area'] / df1_1['area']
df1_1.loc[df1_1['rooms'] == 0, 'ratio'] = df1_1['area']
df1_1['is_last_floor'] = (df1_1['height'] == 1).astype(int)
df1_1['is_first_floor'] = (df1_1['level'] == 1).astype(int)
df1_1['date_difference'] = (pd.Timestamp.now() - pd.to_datetime(df1_1['date'])).dt.days

C:\Users\gromo\AppData\Local\Temp\ipykernel_6856\379415434.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_1['rooms'] = df1_1['rooms'].replace({-1: 0, -2: 0})
C:\Users\gromo\AppData\Local\Temp\ipykernel_6856\379415434.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_1['height'] = df1_1['level'] / df1_1['levels']


In [ ]:
target_point = (59.938962, 30.315586)  # Центр СПБ

def calculate_distance(row, target):
    point = (row['geo_lat'], row['geo_lon'])
    return geodesic(point, target).kilometers

df1_1['distance_to_the_center_km'] = df1_1.apply(calculate_distance, target=target_point, axis=1)

In [59]:
df1_1 = df1_1.drop(['time', 'geo_lat', 'geo_lon', 'region', 'date'], axis=1)

In [60]:
df1_1.corr(method='pearson')

,index,price,building_type,level,levels,rooms,area,kitchen_area,object_type,height,ratio,is_last_floor,is_first_floor,date_difference,distance_to_the_center_km
index,1.000000,0.139845,-0.018600,0.015777,0.018253,-0.014163,-0.032977,0.036701,0.109801,0.002000,0.021077,-0.002235,0.001913,-0.995373,-0.036446
price,0.139845,1.000000,0.076919,-0.076686,-0.177626,0.524687,0.780306,0.490249,-0.021622,0.074389,-0.051603,0.037853,-0.060680,-0.146782,-0.302072
building_type,-0.018600,0.076919,1.000000,-0.071695,-0.127953,0.057542,0.092684,0.042651,-0.028865,0.028075,0.018354,0.039449,0.010674,0.027332,-0.053421
level,0.015777,-0.076686,-0.071695,1.000000,0.600008,-0.136097,-0.095341,0.049633,0.145596,0.652562,0.019352,0.141556,-0.307207,-0.015718,0.019005
levels,0.018253,-0.177626,-0.127953,0.600008,1.000000,-0.227038,-0.170682,0.105271,0.289505,-0.108328,0.025282,-0.188883,-0.193806,-0.018063,0.020336
rooms,-0.014163,0.524687,0.057542,-0.136097,-0.227038,1.000000,0.799798,0.196614,-0.135050,0.041168,-0.215758,0.065452,0.035315,0.013496,-0.088225
area,-0.032977,0.780306,0.092684,-0.095341,-0.170682,0.799798,1.000000,0.461687,-0.082777,0.046091,-0.085627,0.048162,-0.016696,0.032353,-0.156414
kitchen_area,0.036701,0.490249,0.042651,0.049633,0.105271,0.196614,0.461687,1.000000,0.295135,-0.024674,-0.039722,-0.049008,-0.102539,-0.039310,-0.180620
object_type,0.109801,-0.021622,-0.028865,0.145596,0.289505,-0.135050,-0.082777,0.295135,1.000000,-0.066966,0.028265,-0.084380,-0.113875,-0.102683,-0.116745
height,0.002000,0.074389,0.028075,0.652562,-0.108328,0.041168,0.046091,-0.024674,-0.066966,1.000000,0.001359,0.498491,-0.363156,-0.001971,0.007991


In [61]:
encoder1 = ce.BinaryEncoder(cols=['building_type',
                                  'level',
                                    'levels',
                                      'rooms',
                                        'object_type'],
                                          drop_invariant=True,
                                            handle_unknown='ignore')
df1_1_encoded = encoder1.fit_transform(df1_1)

In [62]:
# Убираем мою квартиру из данных

df1_1_encoded_copy = df1_1_encoded.copy()
df1_1_encoded_copy = df1_1_encoded_copy.drop(['index'], axis=1)
my_house_encoded = df1_1_encoded_copy[df1_1_encoded_copy['price'] == 17030506]
df1_1_encoded_copy = df1_1_encoded_copy[df1_1_encoded_copy['price'] != 17030506]
my_house_encoded = my_house_encoded.drop(['price'], axis=1) # Без линеаризации данных

In [63]:
scaler1 = StandardScaler()
df1_1_to_scale = df1_1_encoded.copy()
df1_1_to_scale[['area', 
                'kitchen_area', 
                'ratio', 
                'date_difference', 
                'height',
                'distance_to_the_center_km']] = scaler1.fit_transform(df1_1_to_scale[['area',
                                                                                       'kitchen_area',
                                                                                         'ratio',
                                                                                           'date_difference',
                                                                                             'height',
                                                                                             'distance_to_the_center_km']])


In [64]:
# Убираем мою квартиру из данных
df1_1_to_scale = df1_1_to_scale.drop(['index'], axis=1)
my_house = df1_1_to_scale[df1_1_to_scale['price'] == 17030506]
df1_1_to_scale = df1_1_to_scale[df1_1_to_scale['price'] != 17030506]
my_house_scaled = my_house.drop(['price'], axis=1) # С линеаризованными данными

In [65]:
X1, y1 = df1_1_to_scale.drop(['price'], axis=1), df1_1_to_scale['price']

X1_train, X1_test, y1_train, y1_test = train_test_split(
    X1, y1, test_size=0.2, random_state=42)

In [66]:
ridge_model_1 = Ridge()

In [67]:
ridge_model_1.fit(X1_train, y1_train)

Ridge()

In [68]:
ridge_model_1.coef_

array([ 1187206.59257494,  1065580.79908033,   500974.04250167,
          13403.09208758,  -689580.05320931,  -220450.59651962,
        -151994.12852302,   -94520.16923681,  -219148.52813864,
        -169951.70591721,  -176399.50952876,   -96881.60873691,
        -384140.45421899,    51755.48356182,   703322.53783719,
       -6723805.83726727, -1631444.50635923,   758768.58080831,
        -987940.00489127,  4103797.87627166,   510204.32254099,
        -250357.18263658,   250357.1826239 ,   395056.5183277 ,
         340178.60117988,  -247184.00663524,   250897.15433475,
        -826391.04776207,  -794261.49387248])

In [69]:
ridge_model_1.feature_names_in_

array(['building_type_0', 'building_type_1', 'building_type_2', 'level_0',
       'level_1', 'level_2', 'level_3', 'level_4', 'level_5', 'levels_0',
       'levels_1', 'levels_2', 'levels_3', 'levels_4', 'levels_5',
       'rooms_0', 'rooms_1', 'rooms_2', 'rooms_3', 'area', 'kitchen_area',
       'object_type_0', 'object_type_1', 'height', 'ratio',
       'is_last_floor', 'is_first_floor', 'date_difference',
       'distance_to_the_center_km'], dtype=object)

In [70]:
predicted_y_train1 = ridge_model_1.predict(X1_train)
predicted_y_test1 = ridge_model_1.predict(X1_test)

In [96]:
# rmsle_train1 = root_mean_squared_log_error(y1_train, predicted_y_train1)
# rmsle_test1 = root_mean_squared_log_error(y1_test, predicted_y_test1)
r2_train1 = r2_score(y1_train, predicted_y_train1)
r2_test1 = r2_score(y1_test, predicted_y_test1)
mae_train1 = mean_absolute_error(y1_train, predicted_y_train1)
mae_test1 = mean_absolute_error(y1_test, predicted_y_test1)
mape_train1 = mean_absolute_percentage_error(y1_train, predicted_y_train1)
mape_test1 = mean_absolute_percentage_error(y1_test, predicted_y_test1)
# print("RMSLE Ridge (train): ", rmsle_train1)
print("R2 score Ridge (train): ", r2_train1)
print("MAE score Ridge (train): ", mae_train1)
# print("RMSLE Ridge (test): ", rmsle_test1)
print("R2 score Ridge (test): ", r2_test1)
print("MAE score Ridge (test): ", mae_test1)

print("MAPE score Ridge (train)", mape_train1)
print("MAPE score Ridge (test)", mape_test1)

R2 score Ridge (train):  0.7094153717342342
MAE score Ridge (train):  1629802.4301683172
R2 score Ridge (test):  0.7071648236585806
MAE score Ridge (test):  1642138.910802004
MAPE score Ridge (train) 0.2275470977325669
MAPE score Ridge (test) 0.2271955804683501


In [72]:
xgm_model1 = XGBRFRegressor()

In [73]:
X2, y2 = df1_1_encoded_copy.drop(['price'], axis=1), df1_1_encoded_copy['price']

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.2, random_state=42)

In [74]:
xgm_model1.fit(X2_train, y2_train)

XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=None, device=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, objective='reg:squarederror',
               random_state=None, reg_alpha=None, ...)

In [75]:
xgm_model1.feature_importances_

array([0.0040068 , 0.00425724, 0.02885313, 0.        , 0.00120574,
       0.        , 0.00102389, 0.        , 0.        , 0.        ,
       0.00602503, 0.00478212, 0.00544377, 0.03042928, 0.03672631,
       0.        , 0.00678607, 0.27503392, 0.15694861, 0.27779058,
       0.04105416, 0.00844764, 0.00603315, 0.01005135, 0.03257403,
       0.        , 0.        , 0.01895466, 0.04357255], dtype=float32)

In [76]:
xgm_model1.feature_names_in_

array(['building_type_0', 'building_type_1', 'building_type_2', 'level_0',
       'level_1', 'level_2', 'level_3', 'level_4', 'level_5', 'levels_0',
       'levels_1', 'levels_2', 'levels_3', 'levels_4', 'levels_5',
       'rooms_0', 'rooms_1', 'rooms_2', 'rooms_3', 'area', 'kitchen_area',
       'object_type_0', 'object_type_1', 'height', 'ratio',
       'is_last_floor', 'is_first_floor', 'date_difference',
       'distance_to_the_center_km'], dtype='<U25')

In [77]:
predicted_y_train2 = xgm_model1.predict(X2_train)
predicted_y_test2 = xgm_model1.predict(X2_test)

In [97]:
rmsle_train2 = root_mean_squared_log_error(y2_train, predicted_y_train2)
rmsle_test2 = root_mean_squared_log_error(y2_test, predicted_y_test2)
r2_train2 = r2_score(y2_train, predicted_y_train2)
r2_test2 = r2_score(y2_test, predicted_y_test2)
mae_train2 = mean_absolute_error(y2_train, predicted_y_train2)
mae_test2 = mean_absolute_error(y2_test, predicted_y_test2)
mape_train2 = mean_absolute_percentage_error(y2_train, predicted_y_train2)
mape_test2 = mean_absolute_percentage_error(y2_test, predicted_y_test2)
print("RMSLE XGB (train): ", rmsle_train2)
print("R2 score XGB (train): ", r2_train2)
print("MAE score XGB (train): ", mae_train2)
print("RMSLE XGB (test): ", rmsle_test2)
print("R2 score XGB (test): ", r2_test2)
print("MAE score XGB (test): ", mae_test2)

print("MAPE score XGB (train)", mape_train2)
print("MAPE score XGB (test)", mape_test2)

RMSLE XGB (train):  0.24930342217665227
R2 score XGB (train):  0.7543103694915771
MAE score XGB (train):  1465913.4129418624
RMSLE XGB (test):  0.24978152241038012
R2 score XGB (test):  0.7528921961784363
MAE score XGB (test):  1481342.6448828005
MAPE score XGB (train) 0.19904475744814776
MAPE score XGB (test) 0.19986054348672486


In [79]:
# Предсказание по своему жилью

my_house_xgb_prediction = xgm_model1.predict(my_house_encoded)
my_house_ridge_prediction = ridge_model_1.predict(my_house_scaled)

print('Стоимость жилья по прогнозу моделью Ridge:', my_house_ridge_prediction)
print('Стоимость жилья по прогнозу моделью XGB:', my_house_xgb_prediction)

print('Кадастровая стоимость жилья: 17030506.18 руб.')

Стоимость жилья по прогнозу моделью Ridge: [16598093.67576037]
Стоимость жилья по прогнозу моделью XGB: [13382929.]
Кадастровая стоимость жилья: 17030506.18 руб.


In [80]:
# Самостоятельная работа

df2 = pd.read_csv(r'data/move.csv') # Грузим данные

In [81]:
df2 = df2.drop(['Unnamed: 0'], axis=1) # Убираем дубль индекса

In [82]:
df2 = df2[df2['total_area'] >= (df2['living_area'] + df2['kitchen_area'])] # Проверяем на логику общую площадь
df2 = df2[df2['storeys'] >= df2['storey']] # Проверяем логику предложений на этажность домов

In [83]:
df2['provider'] = df2['provider'].str.strip() # Удаляем в лицо, проводящее сделку, пробелы
df2['metro'] = df2['metro'].str.strip() # Удаляем в метро пробелы
df2['way'] = df2['way'].str.strip() # Удаляем в маршрут пробелы

df2 = df2.drop_duplicates() # Удаляем дубли данных (их очень много)

In [84]:
df2 = df2[(df2['price'] <= (df2['price'].mean() + 3 * df2['price'].std())) & # Удаляем выбросы по ценам
          (df2['price'] >= (df2['price'].mean() - 3 * df2['price'].std()))]

df2 = df2[(df2['views'] <= (df2['views'].mean() + 3 * df2['views'].std())) & # Удаляем выбросы по просмотрам
          (df2['views'] >= (df2['views'].mean() - 3 * df2['views'].std()))]

df2 = df2[(df2['storeys'] <= (df2['storeys'].mean() + 3 * df2['storeys'].std())) & # Удаляем выбросы по этажности
           (df2['storeys'] >= (df2['storeys'].mean() - 3 * df2['storeys'].std()))]

In [85]:
encoder2 = ce.BinaryEncoder(cols=['way',
                                    'provider',],
                                          drop_invariant=True,
                                            handle_unknown='ignore') # Кодируем категории и преобразуем дату
df2_encoded = encoder2.fit_transform(df2)

df2_encoded = pd.get_dummies(df2_encoded,
                             columns=['metro'],
                             prefix='cat').astype(int)

In [86]:
scaler2 = StandardScaler() # Нормализуем значения бинарных признаков и преобразуем дату
df2_to_scale = df2_encoded.copy()
df2_to_scale[['price',
               'views',
                 'fee_percent',
                   'storey',
                     'minutes',
                     'storeys',
                     'living_area',
                     'kitchen_area',
                     'total_area']] = scaler2.fit_transform(df2_to_scale[['price',
               'views',
                 'fee_percent',
                   'storey',
                     'minutes',
                     'storeys',
                     'living_area',
                     'kitchen_area',
                     'total_area']])


In [87]:
X3, y3 = df2_to_scale.drop(['price'], axis=1), df2_to_scale['price'] # Кросс-валидация

X3_train, X3_test, y3_train, y3_test = train_test_split(
    X3, y3, test_size=0.2, random_state=42)

In [88]:
xgm_model2 = XGBRFRegressor() # Нужные модели
random_forest_model = RandomForestRegressor()
extra_trees_model = ExtraTreesRegressor()

In [89]:
xgm_model2.fit(X3_train, y3_train) # Обучение моделей на одних выборках
random_forest_model.fit(X3_train, y3_train)
extra_trees_model.fit(X3_train, y3_train)

ExtraTreesRegressor()

In [90]:
predicted_y_train3 = xgm_model2.predict(X3_train) # Предсказания по каждой из обученных моделей
predicted_y_test3 = xgm_model2.predict(X3_test)

predicted_y_train4 = random_forest_model.predict(X3_train)
predicted_y_test4 = random_forest_model.predict(X3_test)

predicted_y_train5 = extra_trees_model.predict(X3_train)
predicted_y_test5 = extra_trees_model.predict(X3_test)

In [98]:
# Метрики модели XGB

#rmsle_train3 = root_mean_squared_log_error(y3_train, predicted_y_train3)
#rmsle_test3 = root_mean_squared_log_error(y3_test, predicted_y_test3)
r2_train3 = r2_score(y3_train, predicted_y_train3)
r2_test3 = r2_score(y3_test, predicted_y_test3)
mae_train3 = mean_absolute_error(y3_train, predicted_y_train3)
mae_test3 = mean_absolute_error(y3_test, predicted_y_test3)

mape_train3 = mean_absolute_percentage_error(y3_train, predicted_y_train3)
mape_test3 = mean_absolute_percentage_error(y3_test, predicted_y_test3)

#print("RMSLE XGB (train): ", rmsle_train3)
print("R2 score XGB (train): ", r2_train3)
print("MAE score XGB (train): ", mae_train3)
#print("RMSLE XGB (test): ", rmsle_test3)
print("R2 score XGB (test): ", r2_test3)
print("MAE score XGB (test): ", mae_test3)

print("MAPE score XGB (train)", mape_train3)
print("MAPE score XGB (test)", mape_test3)

R2 score XGB (train):  0.8350354364195707
MAE score XGB (train):  0.3025700646458881
R2 score XGB (test):  0.6077418599320041
MAE score XGB (test):  0.4727699729407061
MAPE score XGB (train) 1.2136417622427649
MAPE score XGB (test) 0.923210915192733


In [99]:
# Метрики модели Random forest

#rmsle_train4 = root_mean_squared_log_error(y3_train, predicted_y_train4)
#rmsle_test4 = root_mean_squared_log_error(y3_test, predicted_y_test4)
r2_train4 = r2_score(y3_train, predicted_y_train4)
r2_test4 = r2_score(y3_test, predicted_y_test4)
mae_train4 = mean_absolute_error(y3_train, predicted_y_train4)
mae_test4 = mean_absolute_error(y3_test, predicted_y_test4)

mape_train4 = mean_absolute_percentage_error(y3_train, predicted_y_train4)
mape_test4 = mean_absolute_percentage_error(y3_test, predicted_y_test4)

#print("RMSLE Random forest (train): ", rmsle_train4)
print("R2 score Random forest (train): ", r2_train4)
print("MAE score Random forest (train): ", mae_train4)
#print("RMSLE Random forest (test): ", rmsle_test4)
print("R2 score Random forest (test): ", r2_test4)
print("MAE score Random forest (test): ", mae_test4)

print("MAPE score Random Foreset (train)", mape_train4)
print("MAPE score Random Forest  (test)", mape_test4)

R2 score Random forest (train):  0.907668862447726
MAE score Random forest (train):  0.22085051032090683
R2 score Random forest (test):  0.683516893724122
MAE score Random forest (test):  0.4136621208518772
MAPE score Random Foreset (train) 0.7117149663241381
MAPE score Random Forest  (test) 0.8913649232439722


In [100]:
# Метрики модели Extra trees

#rmsle_train5 = root_mean_squared_log_error(y3_train, predicted_y_train5)
#rmsle_test5 = root_mean_squared_log_error(y3_test, predicted_y_test5)
r2_train5 = r2_score(y3_train, predicted_y_train5)
r2_test5 = r2_score(y3_test, predicted_y_test5)
mae_train5 = mean_absolute_error(y3_train, predicted_y_train5)
mae_test5 = mean_absolute_error(y3_test, predicted_y_test5)
mape_train5 = mean_absolute_percentage_error(y3_train, predicted_y_train5)
mape_test5 = mean_absolute_percentage_error(y3_test, predicted_y_test5)
#print("RMSLE Extra trees (train): ", rmsle_train5)
print("R2 score Extra trees (train): ", r2_train5)
print("MAE score Extra trees (train): ", mae_train5)
#print("RMSLE Extra trees (test): ", rmsle_test5)
print("R2 score Extra trees (test): ", r2_test5)
print("MAE score Extra trees (test): ", mae_test5)

print("MAPE score ExtraTrees (train)", mape_train5)
print("MAPE score ExtraTrees  (test)", mape_test5)



R2 score Extra trees (train):  1.0
MAE score Extra trees (train):  9.683114525258927e-16
R2 score Extra trees (test):  0.4833777659283144
MAE score Extra trees (test):  0.47906874859872856
MAPE score ExtraTrees (train) 1.2529600613247441e-15
MAPE score ExtraTrees  (test) 1.586241768479764


In [94]:
# Самый большой R2 - у модели Random Forest
# Самый маленький MAE также у Random Forest
# RMSLE не проводился из-за отрицательных значений в выборке
# Значения весов по каждой из фичей по модели Random Forest 

random_forest_model.feature_importances_

array([6.71457418e-03, 1.22082629e-02, 1.02685212e-01, 3.04212055e-01,
       1.46498142e-02, 9.37509734e-03, 1.96473540e-02, 4.28109154e-02,
       5.86890702e-02, 4.83623951e-02, 7.61472956e-02, 5.10753089e-02,
       8.63164631e-02, 0.00000000e+00, 5.68797475e-04, 1.56596118e-05,
       4.02845494e-03, 9.39164190e-04, 8.96941553e-05, 7.02236146e-05,
       6.13115050e-04, 1.13424573e-04, 1.32159335e-04, 1.52068812e-03,
       0.00000000e+00, 2.80714981e-04, 1.46318719e-03, 1.47761427e-18,
       0.00000000e+00, 1.70832434e-03, 7.12027571e-05, 1.57159154e-05,
       0.00000000e+00, 5.82823452e-04, 2.09605767e-02, 9.94251169e-04,
       0.00000000e+00, 4.43636290e-04, 7.68232154e-05, 2.64923254e-03,
       2.45933606e-04, 3.09207384e-04, 3.77410025e-03, 3.08704852e-02,
       0.00000000e+00, 1.88537358e-04, 0.00000000e+00, 1.02406459e-03,
       1.96775477e-04, 1.01825208e-05, 2.20501835e-05, 2.22283051e-06,
       2.17791267e-03, 2.71427504e-03, 2.16300095e-03, 5.80628182e-03,
      

In [95]:
random_forest_model.feature_names_in_

array(['way_0', 'way_1', 'views', 'provider_0', 'provider_1',
       'provider_2', 'fee_percent', 'storey', 'minutes', 'storeys',
       'living_area', 'kitchen_area', 'total_area', 'cat_Akademicheskaia',
       'cat_Alekseevskaia', 'cat_Altufevo', 'cat_Annino',
       'cat_Aviamotornaia', 'cat_Bagrationovskaia', 'cat_Beliaevo',
       'cat_Belomorskaia', 'cat_Belorusskaia', 'cat_Borisovo',
       'cat_Bratislavskaia', 'cat_Bulvar Admirala Ushakova',
       'cat_Bulvar Rokossovskogo', 'cat_Buninskaia Alleia',
       'cat_Butyrskaia', 'cat_Cherkizovskaia', 'cat_Chertanovskaia',
       'cat_Dobryninskaia', 'cat_Dubrovka', 'cat_Elektrozavodskaia',
       'cat_Filatov Lug', 'cat_Fili', 'cat_Govorovo',
       'cat_Iugo-Zapadnaia', 'cat_Izmailovskaia', 'cat_Khovrino',
       'cat_Kommunarka', 'cat_Kotelniki', 'cat_Krasnoselskaia',
       'cat_Kuntcevskaia', 'cat_Kurskaia', 'cat_Kuzminki', 'cat_Liublino',
       'cat_Lukhmanovskaia', 'cat_Marino', 'cat_Medvedkovo',
       'cat_Minskaia', 'cat